# Compound Fitting with `SAMMY`

This Jupyter Notebook cell demonstrates the application of the `SAMMY` tool for fitting spectral transmission data using the `pleiades` utilities.

The transmission data utilized in this analysis originates from the [RPI measurement](https://doi.org/10.1016/j.anucene.2014.01.041) of enriched samples of [Eu-nat](https://www-nds.iaea.org/exfor/servlet/X4sGetSubent?subID=14382008) and [Eu-153](https://www-nds.iaea.org/exfor/servlet/X4sGetSubent?subID=14382007), which are documented on [EXFOR](https://www-nds.iaea.org/index-meeting-crp/CM-RF-2013/).

Europium comprises two stable isotopes, Eu-151 and Eu-153, with natural abundances of 47.8% and 52.2%, respectively. This example focuses on determining the abundances of these isotopes in both natural and enriched samples solely based on fitting the resonance transmission spectrum.

The analysis involves two main steps:

1. **Individual Isotope Fitting:** SAMMY is executed separately for each isotope. A simple configuration file is used to generate the resonance parameter file (.par file) employing resonance data directly sourced from ENDF8.

2. **Compound Isotopic Fitting:** The resonance spectra of both isotopes are combined to form an isotopic compound. SAMMY is then run again to fit the resonance spectrum and determine the most probable abundances of the isotopes.